**Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import cv2
from PIL import Image
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten , Dense
from tensorflow.keras.models import Model

**Loading the dataset**

In [ ]:
# Load the Training Dataset
train_dataset_path = "train_labels.csv"
train_dataset = pd.read_csv(train_dataset_path)

# Display the Data of the Dataset
train_dataset.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the List for Images and Targets
train_images  = []
train_labels = []

# Define the Path of the Images
base_path_images = "/content/drive/MyDrive/Cars"

count = 0
# Extract the Images by Iterating
for filename in train_dataset["filename"]:
    # Extract the Image Path
    image_path = os.path.join(base_path_images, filename)

    # Read the Image
    image = Image.open(image_path)

    # Extract the Image Width and Height
    w , h = image.size

    # Get the X,Y,Widt,Height
    x = float(train_dataset["xmin"][count]) / w
    y = float(train_dataset["ymin"][count]) / h
    width  = float(train_dataset["xmax"][count]) / w
    height = float(train_dataset["ymax"][count]) / h


    # Reshape the Image
    reshape_image = image.resize((224,224))

    # Append the Images and Targets
    train_images.append(np.array(reshape_image))
    train_labels.append((x,y,width,height))

    count += 1


**Shape of train images and labels**

In [ ]:
print(f"Shape of the Images is : {np.shape(train_images)}")
print(f"Shape of the Target is : {np.shape(train_labels)}")

In [ ]:
# Convert the images and targets in the Numpy Matrix
Images = np.array(train_images , dtype = "float32") / 255.0
Targets = np.array(train_labels , dtype = "float32")

**Apply Train-Test Split**

In [ ]:





# Split the data
x_train, x_val, y_train, y_val = train_test_split(Images, Targets, test_size=0.1, random_state=42)

# Display the Shapes
print(f"Shape of x_train is: {x_train.shape}")
print(f"Shape of y_train is: {y_train.shape}")
print(f"Shape of x_val is: {x_val.shape}")
print(f"Shape of y_val is: {y_val.shape}")


Shape of x_train is: (202, 224, 224, 3)
Shape of y_train is: (202, 4)
Shape of x_val is: (23, 224, 224, 3)
Shape of y_val is: (23, 4)


**Loading pre_trained Model**

In [ ]:
modelVGG16 = VGG16(weights = "imagenet" , include_top = False , input_shape = (224,224,3))

# Display the Summary of the VGG16 Model
modelVGG16.summary()

**Adding Layers according to need**

In [ ]:
# Freeze the Trainable Layers of VGG16
modelVGG16.trainable = False

# Now Flatten The VGG16 Model Outpust
flatten = Flatten()(modelVGG16.output)

# Now Applying the Fully Connected Layers
x = Dense(256 , activation = "relu")(flatten)
x = Dense(128 , activation = "relu")(x)
x = Dense(64  , activation = "relu")(x)
x = Dense(32  , activation = "relu")(x)
x = Dense(16 , activation = "relu")(x)
bbox = Dense(4)(x)

# Create the FinalModels
finalModel = Model(inputs = modelVGG16.input , outputs = bbox)

# Now Display the Summary of the Final Model
finalModel.summary()

In [ ]:
# Compile the model
finalModel.compile(optimizer='adam', loss='mse')


**describing no. of epochs**

In [ ]:
# Train the model with integer-encoded labels
history = finalModel.fit(x_train , y_train , validation_data = (x_val , y_val) , epochs = 500 , batch_size = 32, verbose = 1)

**Predict model on testing dataset**

In [ ]:
test_dataset_path = "test_labels.csv"

# Read the Test CSV File
test_dataset = pd.read_csv(test_dataset_path)

In [ ]:
# Now get the Images
for filename in test_dataset["filename"]:
    # Get the Image Path
    image_path = os.path.join(base_path_images, filename)

    # Read the Image
    image = Image.open(image_path)

    # Extract the Size of Image
    w,h = image.size

    # Resize the Image
    resize_image = image.resize((224,224))

    # Convert the Numpy Array Image
    cv2_image = np.array(resize_image)

    # Rescale the Image
    cv2_image = cv2_image / 255.0

    # Predict the Image
    finalImage = np.expand_dims(cv2_image,axis=0)
    prediction = finalModel.predict(finalImage)[0]

    # Now get the X,Y,Width and Height
    x = int(prediction[0] * w)
    y = int(prediction[1] * h)
    width = int(prediction[2] * w)
    height = int(prediction[3] * h)

    print(x,y,width,height)


    # Again Create Numpy Array
    cv2_image_2 = np.array(image)
    cv2.rectangle(cv2_image_2 , (x,y) , (width , height) , (255,0,0) , 2)

    # Display the Image
    display(image)
    display(Image.fromarray(cv2_image_2))

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

# Define a directory to save the cropped images
output_dir = "/content/drive/MyDrive/Car/Cropped_Images"
os.makedirs(output_dir, exist_ok=True)
count = 0

# Now get the Images
for filename in test_dataset["filename"]:
    # Get the Image Path
    image_path = os.path.join(base_path_images, filename)
    # Read the Image
    image = Image.open(image_path)
    # Extract the Size of Image
    w, h = image.size
    # Resize the Image
    resize_image = image.resize((224, 224))
    # Convert the Numpy Array Image
    cv2_image = np.array(resize_image)
    # Rescale the Image
    cv2_image = cv2_image / 255.0
    # Predict the Image
    finalImage = np.expand_dims(cv2_image, axis=0)
    prediction = finalModel.predict(finalImage)[0]
    # Now get the X, Y, Width and Height
    x = int(prediction[0] * w)
    y = int(prediction[1] * h)
    width = int(prediction[2] * w)
    height = int(prediction[3] * h)

    # Ensure the coordinates are within the image bounds
    x = max(0, x)
    y = max(0, y)
    width = min(width, w - x)
    height = min(height, h - y)

    # Again Create Numpy Array
    cv2_image_2 = np.array(image)
    cv2.rectangle(cv2_image_2, (x, y), (x + width, y + height), (255, 0, 0), 2)

    # Crop the region defined by the bounding box if it is within bounds
    if width > 0 and height > 0:
        cropped_image = cv2_image_2[y:y + height, x:x + width]

        # Save the cropped image
        cropped_image_path = os.path.join(output_dir, f"cropped_{filename}")
        cv2.imwrite("/content/drive/MyDrive/Car/Cropped_Images/Image{}.jpg".format(count), cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR))
        count += 1

        # Display the Image
        display(image)
        display(Image.fromarray(cv2_image_2))
        display(Image.fromarray(cropped_image))
    else:
        print(f"Skipping {filename} as the bounding box is out of bounds.")


In [ ]:
!apt-get install tesseract-ocr
!pip install pytesseract

In [ ]:
import os
import cv2
import pytesseract
from PIL import Image

# Define the directory containing the cropped images
cropped_images_dir = "/content/drive/MyDrive/Car/Cropped_Images"

# Function to apply OCR on the cropped images
def apply_ocr_on_images():
    for filename in os.listdir(cropped_images_dir):
        if filename.startswith("cropped_"):
            image_path = os.path.join(cropped_images_dir, filename)
            image = Image.open(image_path)
            text = pytesseract.image_to_string(image, lang='eng')
            print(f"Image: {filename}, Extracted Text: {text}")

# Call the function to apply OCR on the cropped images
apply_ocr_on_images()
print('Done!')